In [1]:
import common_datasets.binary_classification as binclas
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler

from mlscorecheck.auc import simplify_roc

from common_datasets.binary_classification import summary_pdf

import numpy as np
import pandas as pd

import tqdm

from config import generate_random_classifier, datasets

In [2]:
N_SAMPLES = 100_000
#N_SAMPLES = 2400
output_file = 'raw-single-100k.csv'

In [3]:
len(datasets)

31

In [4]:
names = [dataset()['name'] for dataset in datasets]

In [5]:
tmp = summary_pdf[summary_pdf['name'].isin(names)].reset_index(drop=True)
tmp = tmp[['name', 'n_col', 'n', 'n_minority', 'imbalance_ratio', 'citation_key']]
tmp['name_key'] = tmp.apply(lambda row: f'{row["name"]} \\cite{{{row["citation_key"]}}}', axis=1)
tmp = tmp[['name_key', 'n', 'n_col', 'n_minority', 'imbalance_ratio']]
tmp.columns = ['name', 'size', 'attr.', 'p', 'imb. ratio']
tmp['n'] = tmp['size'] - tmp['p']
tmp = tmp[['name', 'size', 'attr.', 'p', 'n', 'imb. ratio']]

In [6]:
tmp.index = [idx for idx in range(1, len(tmp)+1)]
print(tmp.to_latex(float_format="%.2f").replace('_', ' '))

\begin{tabular}{llrrrrr}
\toprule
 & name & size & attr. & p & n & imb. ratio \\
\midrule
1 & abalone9 18 \cite{keel} & 731 & 9 & 42 & 689 & 16.40 \\
2 & appendicitis \cite{keel} & 106 & 7 & 21 & 85 & 4.05 \\
3 & australian \cite{keel} & 690 & 16 & 307 & 383 & 1.25 \\
4 & bupa \cite{keel} & 345 & 6 & 145 & 200 & 1.38 \\
5 & CM1 \cite{krnn} & 498 & 21 & 49 & 449 & 9.16 \\
6 & crx \cite{keel} & 653 & 37 & 296 & 357 & 1.21 \\
7 & dermatology-6 \cite{keel} & 358 & 34 & 20 & 338 & 16.90 \\
8 & ecoli1 \cite{keel} & 336 & 7 & 77 & 259 & 3.36 \\
9 & glass0 \cite{keel} & 214 & 9 & 70 & 144 & 2.06 \\
10 & haberman \cite{keel} & 306 & 3 & 81 & 225 & 2.78 \\
11 & hepatitis \cite{krnn} & 155 & 19 & 32 & 123 & 3.84 \\
12 & hypothyroid \cite{krnn} & 3163 & 25 & 151 & 3012 & 19.95 \\
13 & ionosphere \cite{keel} & 351 & 33 & 126 & 225 & 1.79 \\
14 & iris0 \cite{keel} & 150 & 4 & 50 & 100 & 2.00 \\
15 & KC1 \cite{krnn} & 2109 & 21 & 326 & 1783 & 5.47 \\
16 & mammographic \cite{keel} & 830 & 5 & 403 & 42

In [7]:
def acc_sens_spec_at_th(y_test, y_pred, th):
    tp = np.sum((y_pred >= th) & (y_test == 1))
    tn = np.sum((y_pred < th) & (y_test == 0))
    p = np.sum(y_test)
    n = len(y_test) - np.sum(y_test)

    return (tp + tn)/(p + n), tp/p, tn/n

In [8]:
def calculate_scores(y_test, y_pred, y_train, random_state, label=''):
    threshold = np.sum(y_train)/len(y_train)

    acc, sens, spec = acc_sens_spec_at_th(y_test, y_pred, threshold)

    best_ths = []
    best_acc = 0
    for th in np.hstack([np.unique(y_pred), np.array([-np.inf, np.inf])]):
        acc_tmp, _, _ = acc_sens_spec_at_th(y_test, y_pred, th)

        if acc_tmp > best_acc:
            best_acc = acc_tmp
            best_ths = [th]
        elif acc_tmp == best_acc:
            best_ths.append(th)

    best_th = random_state.choice(best_ths)

    best_acc, best_sens, best_spec = acc_sens_spec_at_th(y_test, y_pred, best_th)

    return {
        f'acc{label}': acc,
        f'sens{label}': sens,
        f'spec{label}': spec,
        f'best_acc{label}': best_acc,
        f'best_sens{label}': best_sens,
        f'best_spec{label}': best_spec
    }

In [9]:
import sklearn.datasets
def synthetic_dataset(random_state):
    n_features = random_state.randint(2, 20)
    n_informative = random_state.randint(1, n_features+1)
    if n_informative < n_features:
        n_redundant = random_state.randint(1, n_features - n_informative + 1)
    else:
        n_redundant = 0

    n_clusters_per_class = random_state.randint(1, 2**(n_informative)/2 + 1)
    weights = random_state.random_sample() * 0.8 + 0.1

    X, y = sklearn.datasets.make_classification(
        n_samples=random_state.randint(100, 2000), 
        n_features=n_features,
        n_informative=n_informative, 
        n_redundant=n_redundant, 
        n_repeated=0, 
        n_classes=2, 
        n_clusters_per_class=n_clusters_per_class, 
        weights=(weights, 1 - weights), 
        flip_y=0.01, 
        class_sep=1.0, 
        hypercube=True, 
        shift=0.0, 
        scale=1.0, 
        shuffle=True, 
        random_state=random_state
        )
    
    return {'data': X, 'target': y, 'name': 'synthetic'}


In [10]:
random_state = np.random.RandomState(5)
dropped = 0
results = []

with tqdm.tqdm(total=N_SAMPLES) as bar:
    while len(results) < N_SAMPLES:
        record = {}

        loader = random_state.choice(datasets)
        dataset = loader()
        #dataset = synthetic_dataset(random_state)

        X = dataset['data']
        y = dataset['target']
        name = dataset['name']

        record['dataset'] = name

        if random_state.randint(2) == 0:
            y = 1 - y

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state, stratify=y)

        ss = StandardScaler()
        X_train = ss.fit_transform(X_train)
        X_test = ss.transform(X_test)

        classifier = generate_random_classifier(random_state, p=np.sum(y_train), n=np.sum(1 - y_train))

        bar.set_description("%25s %20s" % (classifier[0].__name__, dataset['name']))
        bar.refresh()

        record['classifier'] = classifier[0].__name__
        record['classifier_params'] = str(classifier[1])

        classifier_obj = classifier[0](**classifier[1])

        classifier_obj.fit(X_train, y_train)

        if classifier[0].__name__ == 'SVC' and classifier_obj.fit_status_ == 1:
            print('failed SVC')
            continue

        record = record | {
            'p': np.sum(y_test), 
            'n': len(y_test) - np.sum(y_test), 
            'p_train': np.sum(y_train), 
            'n_train': len(y_train) - np.sum(y_train)
        }

        y_pred = classifier_obj.predict_proba(X_test)[:, 1]
        y_pred_train = classifier_obj.predict_proba(X_train)[:, 1]

        auc = roc_auc_score(y_test, y_pred)
        auc_train = roc_auc_score(y_train, y_pred_train)

        record = record | {'auc': auc, 'auc_train': auc_train}

        fpr, tpr, thresholds = simplify_roc(*roc_curve(y_test, y_pred))
        fpr_train, tpr_train, thresholds_train = simplify_roc(*roc_curve(y_train, y_pred_train))

        record = record | {
            'fprs': str(fpr.tolist()), 
            'tprs': str(tpr.tolist()),
            'thresholds': str(thresholds.tolist()),
            'n_nodes': len(fpr),
            'fprs_train': str(fpr_train.tolist()), 
            'tprs_train': str(tpr_train.tolist()),
            'thresholds_train': str(thresholds_train.tolist()),
            'n_nodes_train': len(fpr_train)
            }

        if auc < 0.5:
            dropped += 1
            continue

        record = record | calculate_scores(y_test, y_pred, y_train, random_state)
        record = record | calculate_scores(y_train, y_pred_train, y_train, random_state, '_train')

        results.append(record)
        bar.update(1)


     KNeighborsClassifier               SPECTF: 100%|██████████| 100000/100000 [3:58:47<00:00,  6.98it/s]  


In [11]:
dropped

517

In [12]:
data = pd.DataFrame.from_dict(results)

In [13]:
data.iloc[1]

dataset                                                       haberman
classifier                                               XGBClassifier
classifier_params                  {'random_state': 5, 'max_depth': 4}
p                                                                   16
n                                                                   46
p_train                                                             65
n_train                                                            179
auc                                                           0.567935
auc_train                                                     0.994628
fprs                 [0.0, 0.0, 0.06521739130434782, 0.065217391304...
tprs                 [0.0, 0.125, 0.125, 0.1875, 0.1875, 0.25, 0.25...
thresholds           [inf, 0.9796343445777893, 0.8320866823196411, ...
n_nodes                                                             23
fprs_train           [0.0, 0.0, 0.00558659217877095, 0.005586592178...
tprs_t

In [14]:
data.head()

,dataset,classifier,classifier_params,p,n,p_train,n_train,auc,auc_train,fprs,...,spec,best_acc,best_sens,best_spec,acc_train,sens_train,spec_train,best_acc_train,best_sens_train,best_spec_train
0,bupa,RandomForestClassifier,"{'max_depth': 55, 'random_state': 5}",40,29,160,116,0.781466,1.000000,"[0.0, 0.0, 0.06896551724137931, 0.068965517241...",...,0.620690,0.768116,0.950000,0.517241,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,haberman,XGBClassifier,"{'random_state': 5, 'max_depth': 4}",16,46,65,179,0.567935,0.994628,"[0.0, 0.0, 0.06521739130434782, 0.065217391304...",...,0.652174,0.774194,0.125000,1.000000,0.942623,1.000000,0.921788,0.959016,0.846154,1.000000
2,appendicitis,XGBClassifier,"{'random_state': 5, 'max_depth': 2}",4,18,17,67,0.833333,1.000000,"[0.0, 0.0, 0.05555555555555555, 0.055555555555...",...,0.833333,0.909091,0.750000,0.944444,0.976190,1.000000,0.970149,1.000000,1.000000,1.000000
3,wdbc,RandomForestClassifier,"{'max_depth': 136, 'random_state': 5}",72,42,285,170,0.996032,1.000000,"[0.0, 0.0, 0.023809523809523808, 0.02380952380...",...,0.976190,0.973684,0.972222,0.976190,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,saheart,RandomForestClassifier,"{'max_depth': 1, 'random_state': 5}",61,32,241,128,0.713371,0.789127,"[0.0, 0.03125, 0.03125, 0.0625, 0.0625, 0.0937...",...,0.656250,0.698925,0.852459,0.406250,0.699187,0.692946,0.710938,0.756098,0.863071,0.554688


In [15]:
data.to_csv(output_file, index=False)